In [ ]:
#install if not already
#!pip install beautifulsoup4

In [ ]:
from urllib.request import urlopen
 
from urllib.error import HTTPError
 
from urllib.error import URLError
 
from bs4 import BeautifulSoup
import urllib
import datetime as dt
import pandas as pd

 ##  All teams in a dic

In [ ]:
teamsDic = {
    "TelEOS": {"url": "https://www.youtube.com/watch?v=TSo13vorP8g"},
    "YouRoot.io": {"url": "https://www.youtube.com/watch?v=fJJneJJgzwo"},
    "Team Decentralize": {"url": "https://www.youtube.com/watch?v=-Mtxx3IEZLc"},
    "CADChain": {"url": "https://www.youtube.com/watch?v=_aizBkuaXLM"},
    "Way": {"url": "https://www.youtube.com/watch?v=65X7dlDjB3s"},
    "Armada": {"url": "https://www.youtube.com/watch?v=enTh04mX5FQ"},
    "CollectiveHealth": {"url": "https://www.youtube.com/watch?v=itZ4blsBIDU"},
    "DAO Axis": {"url": "https://www.youtube.com/watch?v=-Mtxx3IEZLc"},
    "This Coin Rocks": {"url": "https://www.youtube.com/watch?v=ZR1DdBL8QDE"},
    "EOS-AGE-Project-DNA": {"url": "https://www.youtube.com/watch?v=ZFouTSVoeAQ"},
    "FinCrowds": {"url": "https://youtu.be/IVIGjQkl0NQ"},
    "VIM": {"url": "https://www.youtube.com/watch?v=enTh04mX5FQ"},
    "Undra.io": {"url": "https://www.youtube.com/watch?v=cyL6khy7ipI"},
    "codum": {"url": "https://www.youtube.com/watch?v=sdtqOXH5svg"},
    "iRespo.com": {"url": "https://www.youtube.com/watch?v=NcZZgCa809U"},
    "FairAccess": {"url": "https://www.youtube.com/watch?v=1M_JPKkHhK4"},
    "Delegator": {"url": "https://www.youtube.com/watch?v=cyL6khy7ipI"},
    "DelegationOne": {"url": "https://www.youtube.com/watch?v=8tXweZ9Y1qI"},
    "Magni": {"url": "https://www.youtube.com/watch?v=HcjWpD2Eteg"},
    "Query:": {"url": "https://www.youtube.com/watch?v=HcjWpD2Eteg"},
    "Binary": {"url": "https://www.youtube.com/watch?v=itZ4blsBIDU"},
    "SE5 Focus": {"url": "https://www.youtube.com/watch?v=nAqBqpMRK7E"},
    "Ownit": {"url": "https://www.youtube.com/watch?v=ir6VlVnG8SI"},
    "LoRaChain": {"url": "https://www.youtube.com/watch?v=p0QRzNIKgBs"},
    "Screen Time": {"url": "https://www.youtube.com/watch?v=ZUEhi2LsFCs"},
    "EOSomnia": {"url": "https://www.youtube.com/watch?v=8tXweZ9Y1qI"},
    "Streamer.IO": {"url": "https://www.youtube.com/watch?v=nmZZzxQsIwU"},
    #"AnyChain": {"url": "youku"},
    "payment.one": {"url": "https://www.youtube.com/watch?v=2KG78PGE1yk"}
}

#  MY CODE

## Functions

In [ ]:
def getLikes(url):
    try:
        #load website & parse with bs4
        page = urllib.request.urlopen(url)
        soup = BeautifulSoup(page,'html.parser')
    except URLError as e:
        print(e)
        
        # get likes (unclicked)
    try:
        likes = soup.find_all('button',class_='like-button-renderer-like-button-unclicked')
        return likes[0].text  #TODO: transform to int (int(likes[0].text) )
    except HTTPError as e:
        print(e)
        return '0'

###  Get likes for all urls in teamsDic

In [ ]:
def keepTally(teams):
    for tm in teams:
        # getLikes for team (tm) and add result to new key "likes"
        teams[tm]['likes'] = getLikes(teams[tm]['url'])
        print(tm, teams[tm]['likes'])
    return tm, teams[tm]['likes']        


In [36]:
def keepTally(teams):
    output = {}
    for tm in teams:
        # getLikes for team (tm) and add result to new key "likes"
        output[tm] = getLikes(teams[tm]['url'])
#         print(tm, output[tm])
    return output

# Run the tally

In [37]:
timeRun = dt.datetime.now().strftime("%Hhr%M")
output = keepTally(teamsDic)

TelEOS 2,293
YouRoot.io 3,121
Team Decentralize 1,467
CADChain 1,058
Way 1,058
Armada 2,643
CollectiveHealth 5,390
DAO Axis 1,467
This Coin Rocks 1,491
EOS-AGE-Project-DNA 251
FinCrowds 883
VIM 2,643
Undra.io 1,077
codum 389
iRespo.com 1,135
FairAccess 226
Delegator 1,077
DelegationOne 554
Magni 112
Query: 112
Binary 5,395
SE5 Focus 58
Ownit 20
LoRaChain 1,953
Screen Time 10
EOSomnia 554
Streamer.IO 10
payment.one 



###  Create filename

In [34]:
outputFileName = 'Run_'+timeRun+'.txt'
print(outputFileName)

Run_04hr26.txt


### save to a file

In [40]:
with open(outputFileName, "w") as text_file:
    text_file.write(str(output))

In [ ]:
# import csv
# with open(outputFileName, 'wb') as csvfile:
#     wr = csv.writer(csvfile, quoting=csv.QUOTE_ALL)
#     wr.writerow(result)

In [ ]:
import pandas as pd

In [39]:
output

{'Armada': '2,643',
 'Binary': '5,395',
 'CADChain': '1,058',
 'CollectiveHealth': '5,390',
 'DAO Axis': '1,467',
 'DelegationOne': '554',
 'Delegator': '1,077',
 'EOS-AGE-Project-DNA': '251',
 'EOSomnia': '554',
 'FairAccess': '226',
 'FinCrowds': '883',
 'LoRaChain': '1,953',
 'Magni': '112',
 'Ownit': '20',
 'Query:': '112',
 'SE5 Focus': '58',
 'Screen Time': '10',
 'Streamer.IO': '10',
 'Team Decentralize': '1,467',
 'TelEOS': '2,293',
 'This Coin Rocks': '1,491',
 'Undra.io': '1,077',
 'VIM': '2,643',
 'Way': '1,058',
 'YouRoot.io': '3,121',
 'codum': '389',
 'iRespo.com': '1,135',
 'payment.one': ''}

In [ ]:
tally.to_csv(outputFileName, sep='\t', encoding='utf-8'))

#  ----- TESTS -----

### Get like for single url

In [ ]:
print(getLikes("https://www.youtube.com/watch?time_continue=14&v=hEQAMeaYCr8"))

###  Trial 1

In [ ]:
try:
    html = urlopen("https://www.youtube.com/watch?time_continue=14&v=hEQAMeaYCr8")
except HTTPError as e:
    print(e)
except URLError:
    print("Server down or incorrect domain")
else:
    res = BeautifulSoup(html.read(),"html5lib")
    tags = res.findAll("h1")
    print("len: ",len(tags))
    for tag in tags:
        print(tag)
        print(tag.getText())

###  Trial 2

In [ ]:
url = "https://www.youtube.com/watch?time_continue=14&v=hEQAMeaYCr8"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page,'html.parser')

print(soup.prettify())

In [ ]:
try:
#     a = soup.find_all("button", )
    a = soup.find_all('button',class_='like-button-renderer-like-button-unclicked')
    print("len is : ", len(a))
except HTTPError as e:
    print(e)

In [ ]:
for i in a:
    print(i.text)

In [ ]:
a[0].text